In [1]:
from aspect_analysys import AspectAnalyser

reviews = [
        "Отличный товар, очень доволен покупкой!",
        "Доставка была задержана, но в целом все хорошо.",
        "Доставка ужасно долгая, но обслуживание отличное.",
        "Обслуживание ужасное."
    ]
    
categories = ["товар", "обслуживание", "доставка", "цена", "качество", "интерфейс", "другое"]
sentiments = ["положительный", "нейтральный", "отрицательный"]

analyzer = AspectAnalyser()
results = analyzer.aspect_analysis(reviews, categories, sentiments)
stats = analyzer.stats_analysis(results)

print(stats)

{'товар': {'count': 1, 'sentiments': {'положительный': 1}}, 'качество': {'count': 1, 'sentiments': {'положительный': 1}}, 'доставка': {'count': 2, 'sentiments': {'нейтральный': 1, 'отрицательный': 1}}, 'обслуживание': {'count': 2, 'sentiments': {'положительный': 1, 'отрицательный': 1}}}


In [2]:
import pandas as pd


sentiment_labels = ['положительный', 'отрицательный', 'нейтральный']

# Формируем строки
rows = []
for category, values in stats.items():
    row = {'category': category, 'всего': values['count']}
    for sentiment in sentiment_labels:
        row[sentiment] = values['sentiments'].get(sentiment, 0)
    rows.append(row)

# Создание датафрейма
df = pd.DataFrame(rows)

# Упорядочим столбцы
df = df[['category', 'положительный', 'отрицательный', 'нейтральный', 'всего']]

# Печать
print(df)

       category  положительный  отрицательный  нейтральный  всего
0         товар              1              0            0      1
1      качество              1              0            0      1
2      доставка              0              1            1      2
3  обслуживание              1              1            0      2


In [8]:
import pandas as pd
from langchain_ollama.llms import OllamaLLM


df = pd.DataFrame({
    "category": ["товар", "качество", "доставка", "обслуживание"],
    "положительный": [6, 1, 0, 1],
    "отрицательный": [2, 0, 1, 4],
    "нейтральный": [0, 0, 1, 0],
    "всего": [8, 1, 2, 5]
})

table_text = df.to_string(index=False)
table_text

llm = OllamaLLM(model="llama3")

# Создаем промпт
prompt = f"""
Вот таблица со статистикой анализа отзывов по категориям и тональностям:

{table_text}

Сделай краткое саммари на русском языке: какие категории чаще всего упоминаются, и опиши каждую категорию по тональности.
"""

# Запрос к модели
response = llm.invoke(prompt)
print(response)

Согласно таблице, наиболее часто упоминаемыми категориями являются:

* Товар (8 отзывов) - имеет преимущественно положительный тон (6 из 8), что указывает на высокое удовлетворение клиентов товарами компании.
* Обслуживание (5 отзывов) - имеет преимущественно отрицательный тон (4 из 5), что свидетельствует о неудовлетворенности клиентов качеством обслуживания.

Категория "Доставка" также упоминается, но в меньшей степени (2 отзыва). Её тональность нейтральная (1 положительный и 1 отрицательный отзыв), что может указывать на средне-непритязливое отношение клиентов к доставке.

Категория "Качество" упоминается только один раз, с положительной тональностью. Это может быть связано с тем, что высококачественный товар или услуга вызывает у клиента только положительные эмоции и не оставляет впечатления о качестве обслуживания.

В целом, анализ отзывов показал, что компании в основном хвалятся товарами и критикуют обслуживание.
